## Apartado 1
Se debe implementar un sistema que permita recibir una expresión como entrada (será una expresión
formada solo por minúsculas y sin los signos de puntuación mencionados (los seis signos con los que
vamos a trabajar en este ejercicio y que serán siempre, el punto, la coma, los dos puntos, el punto y
coma, el signo de cierre de interrogación y de exclamación: `.,;:?!`), y la salida será la misma
expresión pero con los cambios correspondientes a la introducción de mayúsculas y signos de
puntuación indicados.
A un nivel alto de especificación podremos considerar que este método tiene esta signatura:
`string addPunctuationBasic(string)`

Es decir recibirá como entrada un string y devolverá como salida un string.
Como primera versión de esta función addPunctuationBasic se implementará un modelo que
simplemente cambia la primera letra por mayúscula y añade al final del string de entrada un punto.
Por ejemplo, al ejecutar

`addPunctuationBasic(“it can be a very complicated thing the ocean”)`
se obtendrá

>It can be a very complicated thing the ocean.

In [2]:
import sys
# !conda install --yes --prefix {sys.prefix} numpy
libraryList = !{sys.executable} -m pip list
if len(list(filter(lambda x: 'numpy ' in x, libraryList))) == 0:
    !{sys.executable} -m pip install numpy
if len(list(filter(lambda x: 'tqdm ' in x, libraryList))) == 0:
    !{sys.executable} -m pip install tqdm
if len(list(filter(lambda x: 'sklearn ' in x, libraryList))) == 0:
    !{sys.executable} -m pip install sklearn
    
import numpy as np


In [3]:
# Cargamos corpus de test
with open('PunctuationTask.test.en') as f:
    PunctuationTaskTestEn = f.readlines()
# Cargamos corpus de check
with open('PunctuationTask.check.en') as f:
    PunctuationTaskCheckEn = f.readlines()
# Cargamos corpus de training data
with open('PunctuationTask.train.en', encoding='utf-8') as f:
    PunctuationTaskTrainEn = f.readlines()

In [4]:
# Verificamos
print(PunctuationTaskTestEn[0])
print(PunctuationTaskCheckEn[0])
print(PunctuationTaskTrainEn[0])

it can be a very complicated thing the ocean 

It can be a very complicated thing, the ocean. 

And it can be a very complicated thing, what human health is. 



In [5]:
def addPunctuationBasic(line):
    # Convertimos el primer caracter de la oración a mayuscula
    first_letter = line[0].upper()
    # Concatenamos el caracter convertido con la oración comenzando desde el segundo caracter a n caracter 
    formatted_line = first_letter + line[1:]
    band = True
    # Tratamiento de puntuacion final operando con posibles espacios a eliminar y el retorno de carro
    if formatted_line[-1] == '\n':
        formatted_line = formatted_line[:-1]
        while band == True:
            band = False
            if formatted_line[-1] == ' ':
                formatted_line = formatted_line[:-1]
                band = True
            else:
                band = False
    formatted_line = formatted_line + '.'
    return formatted_line

In [6]:
# Probamos con un ejemplo y con el corpus de test
frase = 'vamos a probar la puntuación básica'
print(addPunctuationBasic(frase))
print(addPunctuationBasic(PunctuationTaskTestEn[3]))

Vamos a probar la puntuación básica.
We made the ocean unhappy we made people very unhappy and we made them unhealthy.


## Apartado 2
Implementar la función verifyPunctuation con la siguiente signatura
`[(pos,err)] verifyPunctuation(string check, string test)`

Para realizar esta operación se llevará a cabo una tokenización de ambos strings. En este caso se
considerarán tokens todas las secuencias de letras, números y cualquier otro signo que no sea uno de
los seis indicados como signos de puntuación en este ejercicio (`.,;:?!`)
Es decir, esta función devolverá una lista de pares, donde cada par contendrá la posición (indicada
como índice en el string check) y el tipo de error. Los errores posibles son:
* ‘I’ → Insertion
* ‘D’ → Deletion
* ‘S’ → Substitution

Por ejemplo, consideremos que el string de referencia correcto (check) es
“Hello. What’s your name?”
La tokenización generará los siguientes 6 tokens de referencia
* Token 0: Hello
* Token 1: .
* Token 2: What’s
* Token 3: your
* Token 4: name
* Token 5: ?

Consideremos que nuestro algoritmo de puntuación (un caso hipotético para analizar el algoritmo de
verificación) genera la siguiente salida
>“Hello what’s your, name?”

Es decir los tokens generados en este caso son:
* Token 0: Hello
* Token 1: what’s
* Token 2: your
* Token 3: ,
* Token 4: name
* Token 5: ?

Debemos devolver la lista de cambios necesarios para convertir la cadena de tokens generados
(hipótesis) en la cadena de tokens correctos. Para ello, podemos inspirarnos en el algoritmo de la
distancia de Levenshtein. Es importante tener en cuenta que dadas dos cadenas A y B:
> Dist(A,B) == Dist(B,A)

Por lo que podemos abordar el problema tanto desde el punto de los cambios que hay que hacer para
llegar desde la hipótesis hasta el modelo correcto, o bien desde el modelo correco (referencia o check
en la terminología de este ejercicio) hasta la hipótesis generada por nuestro algoritmo de puntuación.
Podemos ver que respecto a nuestro string de referencia (check), el string de test ha ignorado (podemos
decir que borrado respecto al de referencia un ., por tanto tendríamos el error
>(‘D’,1)

En segundo lugar, el token 2 del string de referencia (check) se ha quedado mal en el string de test ya
que en lugar de “What’s” aparece “what’s”. Se trata por tanto de un error de substitución de una palabra
por otra (en este caso por un error de introducción de mayúsculas):
>(‘S’,2)

Y en tercer lugar, entre los tokens ‘your’ y ‘name’ del string correcto (check) se ha introducido un
nuevo token en el string de test, una coma en concreto, por tanto hay un error que calificaríamos como
>(‘I’,4)

Como se puede observar, todos los errores se posicionan (usan los índices) respecto al string correcto
de referencia (check).
Así pues el resultado de

`verifyPunctuation(“Hello. What’s your name?”,`
`“Hello what’s your, name?”)`

sería

> [ (‘D’,1), (‘S’,2), (‘I’, 4) ]


In [7]:
# Funcion para tokenizar
def tokenizador(texto, tokens):
    
    # Verificacion strings con datos
    if len(texto) == 0: return 0
    texto_tokenizado = []
    
    # Generacion de tokens
    start = 0
    for i in range(len(texto)):
        # Espacios
        if texto[i] == ' ':
            texto_tokenizado.append(texto[start:i])
            start=i+1  
        # Analisis de prueba con tokens
        for j in range (len(tokens)):
            # tokens
            if texto[i] == tokens[j]:
                texto_tokenizado.append(texto[start:i])
                start=i
    return texto_tokenizado

In [8]:
# Reference levenshtein https://python-course.eu/applications-python/levenshtein-distance.php
def iterative_levenshtein(s, t):
    """ 
        iterative_levenshtein(s, t) -> ldist
        ldist is the Levenshtein distance between the strings 
        s and t.
        For all i and j, dist[i,j] will contain the Levenshtein 
        distance between the first i characters of s and the 
        first j characters of t
    """

    rows = len(s)+1
    cols = len(t)+1
    dist = [[0 for x in range(cols)] for x in range(rows)]

    # source prefixes can be transformed into empty strings 
    # by deletions:
    for i in range(1, rows):
        dist[i][0] = i

    # target prefixes can be created from an empty source string
    # by inserting the characters
    for i in range(1, cols):
        dist[0][i] = i
        
    for col in range(1, cols):
        for row in range(1, rows):
            if s[row-1] == t[col-1]:
                cost = 0
            else:
                cost = 1
            dist[row][col] = min(dist[row-1][col] + 1,      # deletion
                                 dist[row][col-1] + 1,      # insertion
                                 dist[row-1][col-1] + cost) # substitution
    return dist

In [9]:
# Re-lectura de matriz para detectar camino optimo
def minimal_operations(matriz_levenshtein, check_len, test_len):
  
    # Generando informe
    col = test_len
    row = check_len
    problemas = []
    while col > 1:
        while row > 1:
            # print("col: "+str(col)+" fil: "+str(row)+" max fil col : "+str(check_len)+" "+str(test_len))
            # print("D: "+str(matriz_levenshtein[row-1][col])+" I: "+str(matriz_levenshtein[row][col-1])
            #       +" S: "+str(matriz_levenshtein[row-1][col-1]))
            minimo = min(matriz_levenshtein[row-1][col],    # deletion
                         matriz_levenshtein[row][col-1],   # insertion
                         matriz_levenshtein[row-1][col-1]) # substitution
            # Diagonalizo
            
            if matriz_levenshtein[row-1][col-1] == minimo:
                if matriz_levenshtein[row-1][col-1] == matriz_levenshtein[row][col] - 1:
                    # Substitution
                    problemas.append("S,"+str(row))
                col = col - 1
                row = row - 1
            elif matriz_levenshtein[row-1][col] == minimo:
                # Deletion
                problemas.append("D,"+str(row))
                row = row - 1
            else:
                # Insertion
                problemas.append("I,"+str(row))
                col = col - 1

    return np.flip(problemas)

    

In [10]:
def verifyPunctuation(check, test, tokenizar=True, extended_report=False):
    """ 
        Analizamos distancias según casos posibles combinando situaciones
        Agregue reporte para poder visualizar la matriz en las pruebas
        Agregue tokenizador para simplificar el procesamiento ya que el enunciado no es claro
    """ 
    if(check == '' or test == ''):
        return [(0, 'D'), (0, 'S'), (0, 'I')]
    # Tokens reservados
    reserve_tokens = ['.',',',';',':','?','!']

    if tokenizar:
        # Tokenizamos input
        data_check = tokenizador(check, reserve_tokens)
        data_test = tokenizador(test, reserve_tokens)
    else:
        data_check = check
        data_test = test

    matriz_levenshtein = iterative_levenshtein(data_check, data_test)

    # Generar reporte
    if extended_report:
        # Impresion de verificacion, opcional
        print('DATOS CORRECTOS')
        print(data_check)
        print('DATOS A SER PROBADOS')
        print(data_test)
        
        # Impresion Matriz Levenshtein opcional
        rows = len(data_check)+1
        for r in range(rows):
            print(matriz_levenshtein[r])
            
    # Exportar camino
    return minimal_operations(matriz_levenshtein, len(data_check), len(data_test))

In [11]:
# Prueba de Apartado 2 tokenizada y sin tokenizar
str1 = 'Hello. What’s your name?'
str2 = 'Hello what’s your, name?'

print(verifyPunctuation(str1,str2, tokenizar = False, extended_report = True ))
print(verifyPunctuation(str1,str2, tokenizar = True, extended_report = True))
print(verifyPunctuation("Manhattan","Manahaton"))

DATOS CORRECTOS
Hello. What’s your name?
DATOS A SER PROBADOS
Hello what’s your, name?
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
[1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
[2, 1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]
[3, 2, 1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
[4, 3, 2, 1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
[5, 4, 3, 2, 1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
[6, 5, 4, 3, 2, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
[7, 6, 5, 4, 3, 2, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
[8, 7, 6, 5, 4, 3, 2, 2, 3, 4, 5, 6, 7, 8, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
[9, 8, 7, 6, 5, 4, 3, 3, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
[10, 9, 8, 7, 6, 5, 4, 4, 3, 2, 3, 4, 5, 

In [12]:
# Prueba de corpus tokenizado y sin tokenizar
print(verifyPunctuation(PunctuationTaskCheckEn[3],PunctuationTaskTestEn[3]))

['D,6' 'D,12' 'D,18']


## Apartado 3

Implementar una herramienta que permita recorrer todo el corpus de test y verificación. Es decir, iría
recorriendo una a una las líneas de cada fichero (que están alineadas), aplicaría sobre la frase de test el
algoritmo básico de puntuación (apartado 1: `addPunctuationBasic()` ) y a continuación
comprobaría si el resultado es o no correcto usando la función `verifyPunctuation()` del
apartado 2.

Obtener a continuación los valores relativos a `precisión`, exhaustividad (`recall`) y `F1` para el algoritmo
`addPunctuationBasic()` implementado en el apartado 1.

Consideraremos estos valores como el baseline, el modelo más básico de puntuación que podemos
realizar para estudiar posibles mejoras.

In [43]:
# Cantidad de vees que se acierta dada toda la entrada
def accuracy(tn, fp, fn, tp):
    return (tp + tn) / (tp + tn + fp + fn)

# Cuanto de lo positivo es positivo
def precision(fp, tp):
    return tp / (tp + fp)

# Casos veerdaderos positivos sobre todo lo positivo
def recall_test(fn, tp):
    return tp * 100 / (tp + fn) 

# Correlacion entre precision y recall
def f1_score(recall, precision):
    return 200 * (( recall * precision) / (recall + precision))

In [44]:
from tqdm import tqdm
from sklearn.metrics import confusion_matrix

def calculateMetrics(check, test, punctuationBasic = False):
    
    testProceced = []
    # Obtengo frases de test procesadas con su puntuacion
    # for i in tqdm(range(len(test)),ncols = 100, desc="Agrego puntuacion ...  "):
    #     testProceced.append(addPunctuationBasic(test[i]))
    
    test_analysis = []
    # Obtengo fidelidad del corpus de test
    
    if punctuationBasic:
        for i in tqdm(range(len(test)),ncols = 100 , desc ="Verifico puntuacion ...", disable = False):
            test_analysis.append(verifyPunctuation(check[i], addPunctuationBasic(test[i])))
    else:
        for i in tqdm(range(len(test)),ncols = 100 , desc ="Verifico puntuacion ...", disable = False):
            test_analysis.append(verifyPunctuation(check[i], test[i]))
    
    total = len(test)
    correct = 0
    
    
    # Como los valores verdaderos son de check, asumo todos verdaderos ya que no tengo un algoritmo de prediccion
    y_true = np.ones(len(test))
    
    # Inicio mi vector de prediccion al cual voy a agregarle el resultado de mi verifyOuntuaction
    y_pred = np.zeros(len(test))
    
    for i in tqdm(range(len(test)),ncols = 100 , desc ="Observo ...", disable = False):
        # Si el analysis previo no genero cambios es porque mi AddPuntuactionBasic logro corregir test
        if len(test_analysis[i]) == 0: 
            y_pred[i] = 1
            correct = correct + 1
        
    # analyticslane.com/2019/10/09/numpy-basico-inicializacion-de-arrays-en-numpy/
    # Fabrico mi matriz de confusion para las estadisticas
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    precision_data = precision(fp, tp)
    recall = recall_test(fn, tp)
    return precision_data, recall, f1_score(recall, precision_data) 
    
    # print("Accuracy: "+str(accuracy_data * 100))
    # print("Precision: "+str(precision_data * 100))
    # print("Recall: "+str(recall_data * 100))
    # print("F1 Score: "+str(f1_score(recall, precision_data) * 100))
    # print(correct / total * 100)

In [45]:

precision, recall, f1 = calculateMetrics(PunctuationTaskCheckEn, PunctuationTaskTestEn)
print("Basic Implementation metrics \n Precision: %s \n Recall: %s \n F1: %s \n" % (precision, recall, f1))


Observo ...: 100%|███████████████████████████████████████| 14382/14382 [00:00<00:00, 1308542.05it/s]

Basic Implementation metrics 
 Precision: 1.0 
 Recall: 0.2920317062995411 
 F1: 45.205037132709066 



## Apartado 4

Utilizando el corpus de entrenamiento contenido en PunctuationTask.train.en construir un modelo de
lenguaje inspirado en la idea de 4-gramas. No es exactamente un 4-grama pero está basado en dicho
modelo.
El objetivo de este pseudo 4-grama será predecir si en la posición P de un string debemos introducir un
signo de puntuación (siempre estamos restringiendo el alcance a los seis signos de puntuación
considerados en este ejercicio), o si debemos cambiar la palabra en dicha posición P por mayúscula.
En última instancia, el 4-grama contendrá tuplas de la siguiente forma:


`(token1, token2, token3, operación)`

donde token1, token2 y token3 serán tokens cualesquiera incluidos en el corpus de entrenamiento. Por
tanto estos tokens podrán ser palabras o signos de puntuación, y ‘operación’ será una de las siguientes
operaciones:

`signo de puntuación (que podrá ser uno de los seis considerados: .,;;?!`

`mayúscula (que indica que la siguiente palabra se debe poner en mayúscula)`

`minúscula (que indica que la siguiente palabra deberá estar en minúscula)`

La operación se decide observando el fenómeno más común (frecuencia relativa) de las distintas
operaciones para cada tríada de tokens (token1 token2 token3).
Por ejemplo, podríamos detectar que para la tríada de tokens
`(‘by’, ‘the’, ‘way’)`
la operación más frecuente es insertar el signo de puntuación coma (,)
Una vez creado este modelo de lenguaje se debe implementar una segunda versión de la función que
añade signos de puntuación denominada
`string addPunctuation4gram(string)`

que recibirá como en el apartado 1 un string de entrada, y devolverá el nuevo string con los cambios
introducidos aplicando el modelo de lenguaje previamente entrenado con el 4-grama previamente
indicado.







In [46]:
import re

token_regular_expression = r'[^.,:;?\s]+|[.,:;?]'
character_regular_expression = '([.,;:?!])'


def processTrainData(train_data):
    model_data = ' '.join(train_data).rstrip()
    tokens = re.findall(token_regular_expression, model_data)
    n_grams = []
    for i in range(0, len(tokens)-4):
        j = i+4
        n_grams.append(tokens[i:j])
    train_grams = np.array(n_grams)

    output = {}
    for n_gram in train_grams:
        word_ngram = n_gram[3]
        word_key = ''.join(n_gram[0:3])
        next = ''
        if re.search(character_regular_expression, word_ngram):
            next = 'C ' + word_ngram
        elif word_ngram[0].isupper():
            next = 'M'
        elif word_ngram[0].isupper() != True:
            next = 'm'

        if word_key in output:
            output[word_key] = np.append(output[word_key], next)
        else:
            output[word_key] = np.array([next])

    for word_key in output:
        unique, counts = np.unique(output[word_key], return_counts=True)
        common = np.where(counts == max(counts))
        output[word_key] = unique[common]

    return output

In [47]:
def addPunctuation4gram(test_data, train_data):
    
    train_data = processTrainData(train_data)

    output = []
    for w in tqdm(range(len(test_data)),ncols = 100 , desc =" Entrenando ...", disable = False):
        tokens = re.findall(token_regular_expression, test_data[w])
        i = 0
        while i < len(tokens)-4:
            sentence = tokens[i: i+4]
            items = sentence[0:3]
            key =  ''.join(items)
            if key in train_data:
                value = train_data[key][0]

                if 'C' in value:
                    target_token = tokens[i+3]
                    character =  value.split()[1]
                    if target_token != character:
                        tokens.insert(i+3,character)
                elif 'M' in value:
                    target_token = tokens[i+3]
                    if target_token[0].isupper() != True:
                        tokens[i+3] = target_token.capitalize()
                elif 'm' in value:
                    target_token = tokens[i+3]
                    if target_token[0].isupper():
                        tokens[i+3] = target_token.lower()

            i+=1
            
        transformation = ' '.join(tokens)
        transformation = addPunctuationBasic(transformation)
        output.append(transformation)
    
    return output

punctuation4gram_data = addPunctuation4gram(PunctuationTaskTestEn, PunctuationTaskTrainEn)

 Entrenando ...: 100%|█████████████████████████████████████| 14382/14382 [00:00<00:00, 19632.83it/s]


## Apartado 5

Aplicar el modelo de verificación implementado en el apartado 2, pero contrastando el corpus de
referencia con el resultado generado por el algoritmo de puntuación basado en 4-gramas del apartado 4.
A continuación obtener los valores de precisión, exhaustividad (recall) y F1 sobre estos nuevos
resultados y compararlos con los obtenidos en el apartado 3.
Este nuevo algoritmo de puntuación basado en 4-gramas, ¿mejora los resultados? Analiza si mejora o
empeora, ¿por qué puede ocurrir?

In [ ]:
#Original
# precision, recall, f1 = calculateMetrics(PunctuationTaskCheckEn, PunctuationTaskTestEn)
# print("Basic Implementation metrics \n Precision: %s \n Recall: %s \n F1: %s \n" % (precision, recall, f1))

#4-grams
n_precision, n_recall, n_f1 = calculateMetrics(PunctuationTaskCheckEn, punctuation4gram_data, punctuationBasic = False)
print("4-grams metrics \n Precision: %s \n Recall: %s \n F1: %s \n" % (n_precision, n_recall, n_f1))

Verifico puntuacion ...:  52%|████████████████▋               | 7524/14382 [00:07<00:13, 521.96it/s]

## Apartado 6

Utilizando también ejemplos de TED talks, en un artículo de 2016, Ottokar Tilk y Tanel Alum ha
aplicado un modelo de redes recurrentes bidireccionales para esta misma tarea (restauración de signos
de puntuación en textos no segmentados).
El artículo donde lo describen se encuentra publicado en este enlace:

* https://www.isca-speech.org/archive/Interspeech_2016/pdfs/1517.PDF
* Bidirectional Recurrent Neural Network with Attention Mechanism forPunctuation Restoration (InterSpeech 2016).

El código correspondiente a su implementación se encuentra también disponible en github:

* https://github.com/ottokart/punctuator2

Los resultados de la evaluación para tres signos de puntuación concretos han sido estos (disponibles en
la dirección github anterior):

PUNCTUATION PRECISION RECALL F-SCORE
,COMMA 64.4 45.2 53.1
?QUESTIONMARK 67.5 58.7 62.8
.PERIOD 72.3 71.5 71.9
Overall 68.9 58.1 63.1

El objetivo de este apartado es estudiar la implementación que han hecho estos autores con una red
neuronal y adecuarla al problema que se ha planteado en este ejercicio.

En concreto, este apartado consistirá en la adecuación del modelo de Tilk y Alum al escenario de este
ejercicio. Es decir, aplicar el entrenamiento de la red propuesta por estos autores al corpus de
entrenamiento (PuntuationTask.train.en) y a continuación evaluar el resultado obtenido usando el
modelo de verificación implementado previamente.

Los resultados que obtienes para tu evaluación son similares a los publicados por estos autores.

Nota: Observar el modelo implementado por estos autores en el script error_calculator.py, como
contraste a vuestro algoritmo de verificación (apartado 2) y evaluación (apartados 3 y 5).




## Apartado 7

A partir del trabajo realizado y teniendo en cuenta otros enfoques disponibles en el ámbito de las
tecnologías del lenguaje, investiga qué otros enfoques o estrategias alternativas para esta misma tarea.
El objetivo de este apartado es que busques bibliografía relevante reciente sobre esta tarea, selecciones
uno o dos artículos y describas brevemente el enfoque y resultados obtenidos.

